In [18]:
import numpy as np # Do simulation in numpy to avoid accidentally carrying gradients
import re


In [2]:
# A CA rule which takes 'benzene' and registers it as a transistor
# A rule which takes square and removes them. Maybe remove a random part of them?
# Rule which states a wire must have a specifc shape.
# Rule that a wire mustn't be lonely.

In [3]:
sampleImg = np.array(
    [
        [0, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
    ]
)
sampleImg.shape

(8, 8)

In [4]:


transistorKernel = np.array(
    [
        [1, 1, 1],
        [1, 0, 1],
        [1, 1, 1],
    ]
)

nullKernel = np.array(
    [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
    ]
)

anyKernel =  np.array(
    [
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
    ]
)


def AnyRule(kernel, region):
    return np.any(region * kernel)

def MatchRule(kernel, region):
    return np.array_equal(region, kernel)

def ApplyCA(input, kernel, rule):
    padddedInput = np.pad(input, 1, mode='constant')

    outputArray = np.zeros_like(input)

    for i in range(input.shape[0]):
        for j in range(input.shape[1]):
            region = padddedInput[i:i+3, j:j+3]
            outputArray[i][j] = rule(region, kernel)

    return outputArray

ApplyCA(ApplyCA(sampleImg, transistorKernel, MatchRule), anyKernel, AnyRule)

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [5]:
transitorImg = ApplyCA(ApplyCA(sampleImg, transistorKernel, MatchRule), anyKernel, AnyRule)
wireImg = (sampleImg - transitorImg).clip(0, 1)
wireImg

array([[0, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [6]:
# I want to test
# 2 inputs on left,
# 1 output on right,
# 1 transistor generated.

# Wires can be calculated as a disjoint set, such that connections only require one calculation.

def WireSets(WireImage):
    connections = np.full_like(WireImage, None, dtype=object) 
    WireImage = np.pad(WireImage, pad_width=((1, 0), (1, 0)), mode='constant') # This is only padded to avoid index out of bounds errors.

    def TraceSetEnd(position, newParent):
        # Traces the connections until it reaches an element pointing to itself (the end)
        # Each value iterated over in the process have their parent set to the end value.
        
        while connections[position] is not newParent:
            nextPosition = connections[position]
            connections[position] = newParent
            position = nextPosition

    # Create disjoint sets of connected wires
    for i in range(connections.shape[0]):
        for j in range(connections.shape[1]):
            if WireImage[i + 1, j + 1] == 1: # If current pixel has a wire
                # Check left and up for connections
                
                if WireImage[i, j + 1] == 1: # and the pixel to the left has a wire
                    connections[i, j] = connections[i - 1, j]
                    if WireImage[i + 1, j] == 1: # if both left and up have a wire, set up to use left as parent.
                        TraceSetEnd((i, j - 1), connections[i, j])
                elif WireImage[i + 1, j] == 1: # or the pixel above has a wire
                    connections[i, j] = connections[i, j - 1]
                else: # If no connections found, set as set containing self
                    connections[i, j] = (i, j)

    def TraceSet(position):
        # This should be changed to set values during travel as a dynamic programming approach.
        while position is not connections[position]:
            position = connections[position]
        
        return position

    connectionDict = {}
    setCount = 0
    connectionMap = np.zeros_like(connections)
    for i in range(connections.shape[0]):
        for j in range(connections.shape[1]):
            if connections[i, j] is not None:
                setRepressentative = TraceSet(connections[i, j])

                if setRepressentative not in connectionDict:
                    setCount += 1
                    connectionDict[setRepressentative] = setCount

                connectionMap[i, j] = connectionDict[setRepressentative]

    return connectionMap, setCount

outWireSets, outCount = WireSets(wireImg)
outWireSets

array([[0, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2],
       [0, 0, 0, 0, 0, 0, 0, 2],
       [3, 3, 3, 3, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [7]:
# Ok, now i have a good model for the circuit simulation.
# How should i simulate it?
# I think the transistor map isn't that useful, i should change it to just return a list of transistor positions.
#  No, wait. It is useful, if only for removing wires.
# Because what i want is
# Map -> list of components and connections -> output

In [8]:
# For each component socket
# Query the connection map for what it is connected to

# This has to result in a new dictionary
# wireSet -> connectedCompoenents
# This set can then be translated to
# connectedCompoennets -> per Component socket, other sockets that can be reached.

In [9]:
# Perhaps model transistors as having single pixel sockets

#These are in y, x coordinates. That is annoying. What did i do.
outSockets = [(1,0), (5,0), (4, 7)]
inSockets = [(4, 3), (3, 7), (2,5), (6, 5)]

testOut = outWireSets.copy()
# for pos in inSockets:
#     testOut[pos] = 9


testOut + (transitorImg)* (outCount + 1)

array([[0, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 4, 4, 4, 2],
       [0, 0, 0, 0, 4, 4, 4, 2],
       [3, 3, 3, 3, 4, 4, 4, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [10]:
# I want to have transistors as defined by their center
def GetCAApplication(input, kernel, rule):
    padddedInput = np.pad(input, 1, mode='constant')

    matchList = []
    for i in range(input.shape[0]):
        for j in range(input.shape[1]):
            region = padddedInput[i:i+3, j:j+3]
            if rule(region, kernel) == 1:
                matchList.append((i,j))

    return matchList

transistors = GetCAApplication(sampleImg, transistorKernel, MatchRule)
transistors

[(4, 5)]

In [11]:
# When quereing an array index outside of bounds, returns default value.
class SafeGrid:
    def __init__(self, grid, default=False):
        self.grid = grid
        self.default = default

    def __getitem__(self, index):
        y, x = index
        if 0 <= y < len(self.grid) and 0 <= x < len(self.grid[0]):
            return self.grid[y][x]
        return self.default
    
safeWires = SafeGrid(outWireSets)

In [12]:

ComponentConnections = {}


def addConnectionUtil(wireSet):
    return lambda lst, y, x: lst.append(wireSet[y, x]) if wireSet[y, x] else None

def TransistorConnections(wireSet, transistor): 
    # transistor (y: int, x: int)
    # wireSet should be 2d SafeGrid
    y, x = transistor

    base = []
    collector = []
    emitter = []

    addConnection = addConnectionUtil(wireSet)

    if wireSet[y, x - 2]:
        addConnection(base, y, x - 2)

    if wireSet[y - 2, x]:
        addConnection(collector, y-2, x)
    if wireSet[y + 2, x]:
        addConnection(collector, y + 2, x)
    
    if wireSet[y, x + 2]:
        addConnection(emitter, y, x + 2)

    return base, collector, emitter

testTransistor = TransistorConnections(safeWires, transistors[0])
testTransistor

([], [1], [2])

In [13]:
inpSources = [(1, 0), (5, 0)] 
outSources = [(3, 7)]

def pointConnections(wireSet, point):
    y, x = point
    connections = []
    addConnection = addConnectionUtil(wireSet)

    potentialConnections = [(y-1, x), (y+1, x), (y, x-1), (y, x+1)]
    for yy, xx in potentialConnections:
        addConnection(connections, yy, xx)

    return connections


inpConnections = {f'inp{index}': pointConnections(safeWires, source) for index, source in enumerate(inpSources)}
outConnections = {f'out{index}': pointConnections(safeWires, source) for index, source in enumerate(outSources)}

print(inpConnections)
print(outConnections)


{'inp0': [1], 'inp1': [3]}
{'out0': [2]}


In [14]:
connectionMap = {}

def addToMap(map, key, name):
    if key not in map:
        map[key] = []
    map[key].append(name)

for index, inp in enumerate(inpConnections.keys()):
    for con in inpConnections[inp]:
        addToMap(connectionMap, con, 'inp' + str(index))

for index, out in enumerate(outConnections):
    for con in outConnections[out]:
        addToMap(connectionMap, con, 'out' + str(index))

transistorList = [testTransistor]

for index, transistor in enumerate(transistorList):
    base, collector, emitter = transistor

    for con in base:
        addToMap(connectionMap, con, 'trans' + str(index) + 'base')
    for con in collector:
        addToMap(connectionMap, con, 'trans' + str(index) + 'collector')
    for con in emitter:
        addToMap(connectionMap, con, 'trans' + str(index) + 'emitter')

connectionMap

{1: ['inp0', 'trans0collector'], 3: ['inp1'], 2: ['out0', 'trans0emitter']}

In [15]:
#transistorList = [testTransistor]

transistorDict = {
    'trans0': testTransistor
}


testComponentMap = {**inpConnections, **outConnections, **transistorDict}
print(testComponentMap)

{'inp0': [1], 'inp1': [3], 'out0': [2], 'trans0': ([], [1], [2])}


In [28]:
bool(True + (1 != 1))

True

In [ ]:
#def SimulateCircuit(circuit, operationOrder)
# ^This should contain all the subfuncitons

testOrder1 = ['inp0', 'inp1']
testOrder2 = ['inp0', 'inp1', 'inp1']


def getComponentId(component):
    match = re.match(r"([a-zA-Z]+)(\d*)([a-zA-Z]*)", component)
    if match:
        return match.groups() # prefix, comp_id, comp_type
    return None, None, None

#Component recives signals
def isSink(component):
    prefix, comp_id, comp_type =getComponentId(component)
    if prefix == "out" or comp_type == "base" or comp_type == "collector":
        return True
    return False

#Component emits signals
def isSource(component):
    prefix, comp_id, comp_type =getComponentId(component)
    if prefix == "inp" or comp_type == "emitter":
        return True
    return False


# Update is always flipping state of a source/emitter component.
def UpdateCircuitSource(connectionMap, componentMap, stateDict, source, depth):
    if (depth > 20):
        raise f"Maximum recursion depth of {depth} exceeded"
    
    # Update state of component
    stateDict[source] = not stateDict[source]

    for connection in componentMap[source]:
        componentsToUpdate = [component for component in connectionMap[connection] if isSink(source)]

        if not stateDict[source]:
            # When source is turned off
            wireSources = [component for component in connectionMap[connection] if not isSink(source)]

            if not any(stateDict[source] for source in wireSources):
                #If there is no source that is emitting a signal on the wireSet,
                # update connected components

                updateQueue = set() # Set of sources that are to be flipped.
                for component in componentsToUpdate:
                    stateDict[component] = False

                    prefix, comp_id, comp_type = getComponentId(component)

                    if prefix == 'trans':
                        # When either base or collector is off, turn off emitter (given that it is on)
                        if stateDict[prefix + comp_id + 'emitter']:
                            updateQueue.add(prefix + comp_id + 'emitter')

                for component in updateQueue:
                    stateDict = UpdateCircuitSource(connectionMap, componentMap, stateDict, component, depth + 1)
        else:
            # When source is turned on
            updateQueue = set() # Set of sources that are to be flipped.
            for component in componentsToUpdate:
                stateDict[component] = True

                prefix, comp_id, comp_type = getComponentId(component)

                if prefix == 'trans':
                    if (stateDict[prefix + comp_id + 'base'] * stateDict[prefix + comp_id + 'collector']):
                        if not stateDict[prefix + comp_id + 'emitter']:
                            updateQueue.add(prefix + comp_id + 'emitter')
            
            for component in updateQueue:
                stateDict = UpdateCircuitSource(connectionMap, componentMap, stateDict, component, depth + 1)

    return stateDict





def simulate(connectionMap, componentMap, operationOrder):
    #Return values of output?
    # What is a valid operation order? I was thinking something like x1on, x2on, y1on, x2off, y1off
    # But maybe it is enough descibing the inputs, and then returning the output. 
    # For the AI to tell that the output should have been turned on in between the end, maybe it should be conditioned on multiple orders.

    #Fisrt, get circuit state from componentMap
    stateDict = {}
    for key, value in componentMap.items():
        if isinstance(value, tuple): # If it is a transistor
            stateDict[key] = (False, False, False)
        else: 
            stateDict[key] = False


    def getComponentId(component):
        match = re.match(r"([a-zA-Z]+)(\d*)([a-zA-Z]*)", component)
        if match:
            return match.groups() # prefix, comp_id, comp_type
        return None, None, None

    #Component recives signals
    def isSink(component):
        prefix, comp_id, comp_type =getComponentId(component)
        if prefix == "out" or comp_type == "base" or comp_type == "collector":
            return True
        return False
    
    #Component emits signals
    def isSource(component):
        prefix, comp_id, comp_type =getComponentId(component)
        if prefix == "inp" or comp_type == "emitter":
            return True
        return False


    #Problem, is this guranteed to ever end - No it is not. Can i add a fail safe? like recursion depth mustn't be longer than n steps.
    #simulate single order
    for order in operationOrder:
        # First, flip the order input
        stateDict[order] = not stateDict[order]

        for connection in componentMap[order]:
            for component in connectionMap[connection]:
                ...


    ...

simulate(connectionMap, testComponentMap, testOrder1)

IndentationError: expected an indented block after function definition on line 24 (3342593995.py, line 36)

In [17]:
print(testComponentMap)
print(connectionMap)

{'inp0': [1], 'inp1': [3], 'out0': [2], 'trans0': ([], [1], [2])}
{1: ['inp0', 'trans0collector'], 3: ['inp1'], 2: ['out0', 'trans0emitter']}
